In [8]:
import pandas as pd

# Load the data
data = pd.read_excel('vitrina.xlsx')

# Display the first few rows of the dataset
data.head()

,Phone_new,Number of bills,Average bill,Average number of goods in bill,Revenue,Total quantity,Discount,Number of goods,TN_Автотовары,TN_Аксессуары,...,%TN_Мелкая бытовая техника,%TN_Сделай сам,%TN_Спорт и активный отдых,%TN_ТВ-Аудио,%TN_Товары для дома,%TN_Услуги,"%TN_Хобби, досуг",%TN_Цифровая Техника,%TN_Элитная техника,OrderDate
0,32565748-000000000,8,2469.625,1.0,19757,8,0,8,0,3,...,12.5,0.0,0.0,25.0,0.0,0.0,0,0.0,0,2016-04-26
1,32565750-575448565151 а,1,26599.000,2.0,26599,2,0,2,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-03-31
2,32565750-575448565151 а5,1,1590.000,1.0,1590,1,0,1,0,0,...,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-04-12
3,43553240-575054413251+75,2,5064.500,1.0,10129,2,0,2,0,0,...,50.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-04-09
4,55445748-514957495657715,1,27449.000,1.0,27449,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-04-28


In [9]:
# Rename columns for easier access
data.rename(columns={
    'Phone_new': 'customer_id',
    'OrderDate': 'transaction_date',
    'Revenue': 'transaction_amount'
}, inplace=True)

# Convert the transaction date to datetime format
data['transaction_date'] = pd.to_datetime(data['transaction_date'])

# Display the first few rows to confirm changes
data.head()


,customer_id,Number of bills,Average bill,Average number of goods in bill,transaction_amount,Total quantity,Discount,Number of goods,TN_Автотовары,TN_Аксессуары,...,%TN_Мелкая бытовая техника,%TN_Сделай сам,%TN_Спорт и активный отдых,%TN_ТВ-Аудио,%TN_Товары для дома,%TN_Услуги,"%TN_Хобби, досуг",%TN_Цифровая Техника,%TN_Элитная техника,transaction_date
0,32565748-000000000,8,2469.625,1.0,19757,8,0,8,0,3,...,12.5,0.0,0.0,25.0,0.0,0.0,0,0.0,0,2016-04-26
1,32565750-575448565151 а,1,26599.000,2.0,26599,2,0,2,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-03-31
2,32565750-575448565151 а5,1,1590.000,1.0,1590,1,0,1,0,0,...,100.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-04-12
3,43553240-575054413251+75,2,5064.500,1.0,10129,2,0,2,0,0,...,50.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-04-09
4,55445748-514957495657715,1,27449.000,1.0,27449,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,2016-04-28


In [28]:
# Define a reference date for recency calculation (typically the most recent transaction date + 1 day)
reference_date = data['transaction_date'].max() + pd.DateOffset(1)

# Calculate RFM metrics
rfm = data.groupby('customer_id').agg({
    'transaction_date': lambda x: (reference_date - x.max()).days,
    'transaction_amount': 'sum',
    'Number of bills': 'sum'
}).reset_index()

# Rename columns
rfm.columns = ['customer_id', 'recency', 'monetary', 'frequency']

# Display the RFM table
rfm.head()


,customer_id,recency,monetary,frequency
0,32565748-000000000,5,19757,8
1,32565750-575448565151 а,31,26599,1
2,32565750-575448565151 а5,19,1590,1
3,43553240-575054413251+75,22,10129,2
4,55445748-514957495657715,3,27449,1


In [11]:
# Define scoring functions
def r_score(x):
    if x <= 30:
        return 5
    elif x <= 60:
        return 4
    elif x <= 90:
        return 3
    elif x <= 120:
        return 2
    else:
        return 1

def fm_score(x):
    if x >= 50:
        return 5
    elif x >= 30:
        return 4
    elif x >= 20:
        return 3
    elif x >= 10:
        return 2
    else:
        return 1

# Apply scoring
rfm['r_score'] = rfm['recency'].apply(r_score)
rfm['f_score'] = rfm['frequency'].apply(fm_score)
rfm['m_score'] = rfm['monetary'].apply(fm_score)

# Combine RFM scores into a single score
rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

# Display the scored RFM table
rfm.head()


,customer_id,recency,monetary,frequency,r_score,f_score,m_score,rfm_score
0,32565748-000000000,5,19757,8,5,1,5,515
1,32565750-575448565151 а,31,26599,1,4,1,5,415
2,32565750-575448565151 а5,19,1590,1,5,1,5,515
3,43553240-575054413251+75,22,10129,2,5,1,5,515
4,55445748-514957495657715,3,27449,1,5,1,5,515


In [12]:
def segment_rfm(df):
    if df['rfm_score'] == '555':
        return 'Best Customers'
    elif df['rfm_score'].startswith('5'):
        return 'Loyal Customers'
    elif df['rfm_score'].endswith('5'):
        return 'Big Spenders'
    elif df['r_score'] == 5:
        return 'Recent Customers'
    elif df['f_score'] == 5:
        return 'Frequent Customers'
    elif df['m_score'] == 5:
        return 'High Value Customers'
    else:
        return 'Others'

# Apply segmentation
rfm['segment'] = rfm.apply(segment_rfm, axis=1)

# Display the segmented RFM table
rfm.head()


,customer_id,recency,monetary,frequency,r_score,f_score,m_score,rfm_score,segment
0,32565748-000000000,5,19757,8,5,1,5,515,Loyal Customers
1,32565750-575448565151 а,31,26599,1,4,1,5,415,Big Spenders
2,32565750-575448565151 а5,19,1590,1,5,1,5,515,Loyal Customers
3,43553240-575054413251+75,22,10129,2,5,1,5,515,Loyal Customers
4,55445748-514957495657715,3,27449,1,5,1,5,515,Loyal Customers


In [ ]:
rfm.to_excel('rfm_analysis.xlsx')

In [14]:
from collections import Counter 
print(Counter(rfm['segment']))

Counter({'Big Spenders': 44985, 'Loyal Customers': 43880, 'Best Customers': 16, 'Others': 7})


In [26]:
rfm_data = rfm.groupby(by='segment').mean(numeric_only=True)
rfm_data

,recency,monetary,frequency,r_score,f_score,m_score
segment,,,,,,
Best Customers,2.375000,2.323120e+06,159.875000,5.000000,5.000000,5.000000
Big Spenders,47.157297,9.782145e+03,1.065911,3.968389,1.000200,5.000000
Loyal Customers,15.212534,1.198010e+04,1.232680,5.000000,1.006586,4.999499
Others,50.714286,4.271429e+01,1.000000,4.000000,1.000000,3.857143


In [29]:
rfm.to_excel('rfm_data.xlsx')